In [1]:
import os
import warnings
import numpy as np
import luigi
import emu.neuralynx_io as nlx
from tqdm import tqdm as tqdm
from emu.pdil.raw import Electrophysiology,Participant,get_data_manifest,points_to_choice
from emu.neuralynx_io import nev_as_records
from emu.nwb import nlx_to_nwb
from pynwb import TimeSeries, NWBFile,NWBHDF5IO
from pynwb.ecephys import ElectricalSeries
from pynwb.misc import AnnotationSeries
import pandas as pd
import datetime
import glob

In [2]:
all_files = get_data_manifest()

In [3]:
seeg_root = os.path.expanduser('~/.emu/pdil/pt_01/SEEG/raw/')

nev_path = os.path.expanduser('~/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.Events.nev')
ncs_paths = sorted(glob.glob(os.path.join(seeg_root,'PO_Day_02.*.ncs')))

# ncs = nlx.load_ncs()
print('nev path: ',nev_path)
print('ncs_paths:')
for p in ncs_paths:
    print(p)

nev path:  /home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.Events.nev
ncs_paths:
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC100_0005.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC100_0006.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC100_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC101_0005.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC101_0006.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC101_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC102_0005.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC102_0006.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC102_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC103_0005.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC103_0006.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC103_0007.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC104_0005.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.CSC104_0006.ncs
/home/el

In [4]:
p = Participant(patient_id=1,raw_files=all_files,seeg_raw_path=seeg_root)

In [5]:
# Create a list of download tasks for the POD4 ncs files
all_ncs_tasks = list(p.cache_ncs())
d2_tasks = [t for t in all_ncs_tasks if 'PO_Day_02' in t.output().path and '0007' in t.output().path]

# Create a list of download tasks for the POD4 nev files
d2_nev_task = [t for t in list(p.cache_nev()) if 'PO_Day_02.Events_0007.nev' in t.output().path][0]

In [6]:
d2_nev_task

NLXRaw(data_root=/home/elijahc/.emu/, file_id=632777786111, file_name=PO_Day_02.Events_0007.nev, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1)

In [7]:
# Make sure the files are cached locally
luigi.build(d2_tasks+[d2_nev_task],workers=4)

DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632794395007, file_name=PO_Day_02.CSC100_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632794395007_PO_Day_02_CSC100_14ed24f81f   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632794415407, file_name=PO_Day_02.CSC101_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632794415407_PO_Day_02_CSC101_532df342f7   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632794398607, file_name=PO_Day_02.CSC102_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632794398607_PO_Day_02_CSC102_2de4f

DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632782049453, file_name=PO_Day_02.CSC123_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632782049453_PO_Day_02_CSC123_68789710e7   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632782033853, file_name=PO_Day_02.CSC124_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632782033853_PO_Day_02_CSC124_03e6a8fa98   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632782078253, file_name=PO_Day_02.CSC125_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632782078253_PO_Day_02_CSC125_fec89

DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632779514632, file_name=PO_Day_02.CSC145_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632779514632_PO_Day_02_CSC145_ad82d95506   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632779529032, file_name=PO_Day_02.CSC146_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632779529032_PO_Day_02_CSC146_6d2b36e50a   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632785993033, file_name=PO_Day_02.CSC147_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632785993033_PO_Day_02_CSC147_2f4c2

DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632784428345, file_name=PO_Day_02.CSC167_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632784428345_PO_Day_02_CSC167_742d15419c   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632784458345, file_name=PO_Day_02.CSC168_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632784458345_PO_Day_02_CSC168_11d5d41d52   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632784435545, file_name=PO_Day_02.CSC169_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632784435545_PO_Day_02_CSC169_c623e

DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632788680781, file_name=PO_Day_02.CSC27_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632788680781_PO_Day_02_CSC27__6e4d93fa08   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632794310918, file_name=PO_Day_02.CSC28_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632794310918_PO_Day_02_CSC28__1fce44fcba   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632794294118, file_name=PO_Day_02.CSC29_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632794294118_PO_Day_02_CSC29__611520d6

INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632792039787_PO_Day_02_CSC49__5c1b6086dd   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632791995387, file_name=PO_Day_02.CSC4_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632791995387_PO_Day_02_CSC4_0_d389a4d058   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632792048187, file_name=PO_Day_02.CSC50_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632792048187_PO_Day_02_CSC50__908955f925   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632798628616, file_name=PO_Day_02.CSC51_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, pati

DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632784377233, file_name=PO_Day_02.CSC71_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632784377233_PO_Day_02_CSC71__86b57f2008   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632784374833, file_name=PO_Day_02.CSC72_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632784374833_PO_Day_02_CSC72__1b4fd439ef   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632784370033, file_name=PO_Day_02.CSC73_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632784370033_PO_Day_02_CSC73__eb094568

INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632785727138_PO_Day_02_CSC93__e1ee83a7a7   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632789432163, file_name=PO_Day_02.CSC94_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632789432163_PO_Day_02_CSC94__e0cfbf8088   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632789459763, file_name=PO_Day_02.CSC95_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632789459763_PO_Day_02_CSC95__71aeb1c60b   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632789422563, file_name=PO_Day_02.CSC96_0007.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, pat

True

In [15]:
d2_nev_task.output()
d2_nev = d2_nev_task.output().path
d2_ncs_paths = [t.output().path for t in d2_tasks]

In [16]:
nwb = p.create_nwb(d2_nev,d2_ncs_paths,blocks=[1,2],desc='Patient 1 | Post-op Day 2')

adding to nwb: 0it [00:00, ?it/s]
compressing channels:   0%|          | 0/176 [00:00<?, ?it/s]


3  blocks




compressing channels:  63%|██████▎   | 111/176 [01:50<01:06,  1.02s/it]


compressing channels: 100%|██████████| 176/176 [02:56<00:00,  1.00s/it]
adding to nwb: 176it [02:56,  1.00s/it]
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629615908637, file_name=blockNum_3_computerTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629615908637_blockNum_3_compu_3484a4f2e7   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629620274728, file_name=blockNum_4_humanTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629620274728_blockNum_4_human_73d59337a6   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611713944, file_name=blockNum_5_humanTT_coop_taskoutput.mat, save_to=/ho

In [20]:
nwb.trials.to_dataframe().head()

,start_time,stop_time,outcome
id,,,
0,218.069584,235.699535,C-C
1,256.918818,272.899744,D-C
2,288.068212,305.089857,C-D
3,311.117797,327.435088,C-C
4,331.000757,398.941575,D-C


In [21]:
nwb.acquisition['ttl']

ttl pynwb.misc.AnnotationSeries at 0x139995178238976
Fields:
  comments: no comments
  conversion: 1.0
  data: ['block_start' 'block_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'block_start' 'block_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'block_start' 'block_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start']
  description: no description
  interval: 1
  resolution: -1.0
  timestamps: [ 166.02063608  166.23729205  218.06958389  256.918818    288.06821203
  311.1177969   331.00075698  356.38362193  375.11661506  395.88288689
  491.21444297  491.43109894  510.847404    

In [22]:
with NWBHDF5IO(os.path.join(seeg_root,'..','processed','PO_Day_02.nwb'),'w') as io:
    io.write(nwb)